In [1]:
import cv2
import numpy as np
from operator import itemgetter
import os

In [2]:
# processando a imagem!
def process(img):
     # convercao de RGB para BGR e grayscale
     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
     # aplicando o filtro gaussiano 
     img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
     # detectando as bordas
     img_canny = cv2.Canny(img_blur, 100, 200)
     # retorna um array com o tamanho (9,9 (?)) preenchido com 1
     kernel = np.ones((9, 9))
     # dilatando a imagem
     img_dilate = cv2.dilate(img_canny, kernel, iterations=1)
     # kernel = np.ones((5,5))
     # erodindo a imagem
     return cv2.erode(img_dilate, kernel, iterations=1)

def process_symbols(img):
     kernel = np.ones((3, 3))
     img_erode =  cv2.erode(img, kernel, iterations=1)
     img_dilate = cv2.dilate(img_erode, kernel, iterations=1)
     return img_dilate

# achando os contornos!
def convex_hull(cnt):
     # distancia maxima do contorno pra um contorno aproximado, eh o que garante um bom contorno (eh um parametro de precisao)
     # calcula o perimetor do contorno!
     epsilon = cv2.arcLength(cnt, True)
     # aproximacao dos poligonos com a precisao do epsilon
     approx = cv2.approxPolyDP(cnt, epsilon * 0.02, True)
     # corrige uma "curva" defeituosa do contorno
     return cv2.convexHull(approx).squeeze()

# recebe
def get_square_corners_v2(contours):
     flag = 0 
     for contour in contours:
          if len(contour)<4:
               continue
          inner = contour
          ordenando_indice_x = inner[...,0].argsort()
          ordenando_x_menor = sorted(inner[ordenando_indice_x], key=itemgetter(0))

          lista_x_menor = np.array(ordenando_x_menor[0:2])
          lista_x_maior = np.array(ordenando_x_menor[2:])

          ordenando_indice_y_menor = lista_x_menor[...,1].argsort()
          top_lef_inner, bot_lef_inner = sorted(lista_x_menor[ordenando_indice_y_menor], key=itemgetter(1))

          ordenando_indice_y_maior = lista_x_maior[...,1].argsort()
          top_rit_inner, bot_rit_inner = sorted(lista_x_maior[ordenando_indice_y_maior], key=itemgetter(1))

          dist = np.linalg.norm(top_rit_inner - top_lef_inner)
          if dist > 100:
               flag = True
               break
     
     if not flag:
          return None
               
     p2 = 2*top_lef_inner - bot_lef_inner
     p5 = 2*top_lef_inner - top_rit_inner
     p3 = 2*top_rit_inner - bot_rit_inner
     p6 = 2*top_rit_inner - top_lef_inner
     p7 = 2*bot_lef_inner - bot_rit_inner
     p10 = 2*bot_lef_inner - top_lef_inner
     p8 = 2*bot_rit_inner - bot_lef_inner
     p11 = 2*bot_rit_inner - top_rit_inner
     p1 = 2*top_lef_inner - bot_rit_inner
     p4 = 2*top_rit_inner - bot_lef_inner
     p9 = 2*bot_lef_inner - top_rit_inner
     p12 = 2*bot_rit_inner - top_lef_inner

     return [[p1,p2,top_lef_inner,p5],[p2,p3,top_rit_inner,top_lef_inner],[p3,p4,p6,top_rit_inner],[p5,top_lef_inner,bot_lef_inner,p7],[top_lef_inner,top_rit_inner,bot_rit_inner,bot_lef_inner],[top_rit_inner,p6,p8,bot_rit_inner],[p7,bot_lef_inner,p10,p9],[bot_lef_inner,bot_rit_inner,p11,p10],[bot_rit_inner,p8,p12,p11]]
     
def get_square_corners(inner, outer): 

     ordenando_indice_x = inner[...,0].argsort()
     ordenando_x_menor = sorted(inner[ordenando_indice_x], key=itemgetter(0))

     lista_x_menor = np.array(ordenando_x_menor[0:2])
     lista_x_maior = np.array(ordenando_x_menor[2:])

     ordenando_indice_y_menor = lista_x_menor[...,1].argsort()
     top_lef_inner, bot_lef_inner = sorted(lista_x_menor[ordenando_indice_y_menor], key=itemgetter(1))
     
     ordenando_indice_y_maior = lista_x_maior[...,1].argsort()
     top_rit_inner, bot_rit_inner = sorted(lista_x_maior[ordenando_indice_y_maior], key=itemgetter(1))

     # pegando todas as linhas e coluna 0, e fazendo um sort com os indices (da coluna do eixo X)! (ordem crescente)
     sort_outer_index_x0 = outer[..., 0].argsort()
     # pegando todas as linhas e coluna 1, e fazendo um sort com os indices (da coluna do eixo Y)! (ordem crescente)
     sort_outer_index_y1 = outer[..., 1].argsort()

     # definindo os pontos laterais
     ordenando_x_menor = sorted(outer[sort_outer_index_x0], key=itemgetter(0))
     ordenando_y_menor = sorted(outer[sort_outer_index_y1], key=itemgetter(1))

     lista_x_menor = np.array(ordenando_x_menor[0:2])
     lista_x_maior = np.array(ordenando_x_menor[6:])

     ordenando_indice_y_menor = lista_x_menor[...,1].argsort()

     left_top_outer, left_bot_outer = sorted(lista_x_menor[ordenando_indice_y_menor], key=itemgetter(1))
     
     ordenando_indice_y_maior = lista_x_maior[...,1].argsort()
     rig_top_outer, rig_bot_outer = sorted(lista_x_maior[ordenando_indice_y_maior], key=itemgetter(1))

     # definindo
     lista_y_menor = np.array(ordenando_y_menor[0:2])
     lista_y_maior = np.array(ordenando_y_menor[6:])

     ordenando_indice_x_menor = lista_y_menor[...,1].argsort()
     top_left_outer, top_rit_outer = sorted(lista_y_menor[ordenando_indice_x_menor], key=itemgetter(0))
     
     ordenando_indice_x_maior = lista_y_maior[...,1].argsort()
     bot_left_outer, bot_rit_outer = sorted(lista_y_maior[ordenando_indice_x_maior], key=itemgetter(0))


     # calculando os pontos que nao temos
     top_left_zero = top_left_outer + left_top_outer - top_lef_inner 
     bot_left_zero = bot_left_outer + left_bot_outer - bot_lef_inner
     top_rit_zero = top_rit_outer + rig_top_outer - top_rit_inner
     bot_rit_zero = bot_rit_outer + rig_bot_outer - bot_rit_inner
     # yield np.mean ([bot_rit_outer],0)

     # yield np.mean ([top_left_zero],0)
     # yield np.mean ([top_lef_inner],0)
     # yield np.mean ([left_top_outer],0)

     quadrado_11 = [top_left_zero, top_left_outer, top_lef_inner, left_top_outer]
     quadrado_12 = [top_left_outer, top_rit_outer, top_rit_inner, top_lef_inner]
     quadrado_13 = [top_rit_outer, top_rit_zero, rig_top_outer, top_rit_inner]
     quadrado_21 = [left_top_outer, top_lef_inner, bot_lef_inner, left_bot_outer]
     quadrado_22 = [top_lef_inner, top_rit_inner, bot_rit_inner, bot_lef_inner]
     quadrado_23 = [top_rit_inner, rig_top_outer, rig_bot_outer, bot_rit_inner]
     quadrado_31 = [left_bot_outer, bot_lef_inner, bot_left_outer, bot_left_zero]
     quadrado_32 = [bot_lef_inner, bot_rit_inner, bot_rit_outer, bot_left_outer]
     quadrado_33 = [bot_rit_inner, rig_bot_outer, bot_rit_zero, bot_rit_outer]

     # yield np.mean(quadrado_11,0)
     # yield np.mean(quadrado_12,0)
     # yield np.mean(quadrado_13,0)
     # yield np.mean(quadrado_21,0)
     # yield np.mean(quadrado_22,0)
     # yield np.mean(quadrado_23,0)
     # yield np.mean(quadrado_31,0)
     # yield np.mean(quadrado_32,0)
     # yield np.mean(quadrado_33,0)

     quadrados = [quadrado_11, quadrado_12, quadrado_13, quadrado_21, quadrado_22, quadrado_23, quadrado_31, quadrado_32, quadrado_33]

     return quadrados

# recebe os 4 vertices para cortar na imagem      
def get_squares_init(pontos_quadrado):

     # https://stackoverflow.com/questions/48301186/cropping-concave-polygon-from-image-using-opencv-python

     # criando um retangulo com os vertices
     rect_corte = cv2.boundingRect(pontos_quadrado)
     # passando as coordenadas para as variaveis
     x,y,w,h = rect_corte
     # cortando a imagem com as variaveis
     croped = img_processada[y:y+h, x:x+w].copy()

     # fazendo mascara para o corte da imagem
     pontos_quadrado = pontos_quadrado - pontos_quadrado.min(axis=0)
     mask = np.zeros(croped.shape[:2], np.uint8)
     # desenhando os contornos usando a mascara
     cv2.drawContours(mask, [pontos_quadrado], -1, (255, 255, 255), -1, cv2.LINE_AA)

     # fazendo comparacao da imagem cortada com a mascara
     croped_image_final = cv2.bitwise_and(croped, croped, mask=mask)
     return croped_image_final
     # cv2.imshow("final",croped_image_final)
     # cv2.waitKey(0)
     
def get_squares_after_play(pontos_quadrado, img):

     # https://stackoverflow.com/questions/48301186/cropping-concave-polygon-from-image-using-opencv-python

     # criando um retangulo com os vertices
     rect_corte = cv2.boundingRect(pontos_quadrado)
     # passando as coordenadas para as variaveis
     x,y,w,h = rect_corte
     # cortando a imagem com as variaveis
     croped = img[y:y+h, x:x+w].copy()

     # fazendo mascara para o corte da imagem
     pontos_quadrado = pontos_quadrado - pontos_quadrado.min(axis=0)
     mask = np.zeros(croped.shape[:2], np.uint8)
     # desenhando os contornos usando a mascara
     cv2.drawContours(mask, [pontos_quadrado], -1, (255, 255, 255), -1, cv2.LINE_AA)

     # fazendo comparacao da imagem cortada com a mascara
     croped_image_final = cv2.bitwise_and(croped, croped, mask=mask)
     return croped_image_final
     # cv2.imshow("final",croped_image_final)
     # cv2.waitKey(0)
        
# recebe a imagem sem filtrar (imagem original dos quadrados com os simbolos)
def get_circles(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    
    gray_blurred = cv2.blur(gray, (3, 3)) 
    
    # Apply Hough transform on the blurred image. 
    detected_circles = cv2.HoughCircles(gray_blurred,  
                    cv2.HOUGH_GRADIENT, 1, 20, param1 = 50, 
                param2 = 30, minRadius = 1, maxRadius = 40) 
    
    # desenhando os circulos detectados (precisamos desenhar msm, n eh soh pegar se tem ou n?)
    if detected_circles is not None: 
    # Convert the circle parameters a, b and r to integers. 
        detected_circles = np.uint16(np.around(detected_circles)) 
    
        for pt in detected_circles[0, :]: 
            a, b, r = pt[0], pt[1], pt[2] 
    
            # Draw the circumference of the circle. 
            cv2.circle(img, (a, b), r, (0, 255, 0), 2) 
    
            # Draw a small circle (of radius 1) to show the center. 
            cv2.circle(img, (a, b), 1, (0, 0, 255), 3) 
            cv2.imshow("Detected Circle", img) 
            
            cv2.waitKey(0) 

# Cria do Dataset de Círculos

In [6]:
index_image = 0
defective_images = []
for f in range(1,20):
     img = cv2.imread(f"Webcam/O/TabuleiroO{f}.jpg")
     img_processada = process(img)
     contours, _ = cv2.findContours(img_processada, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE) #talvez usar o CHAIN_APPROX_SIMPLE, ver a diferenca em https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html
     aux = sorted(map(convex_hull, contours), key=len)
     lista_quadrados = get_square_corners_v2(aux)
     
     for file in os.listdir('Webcam/O/'):
          if file.find(f'O{f}-') != -1:
               img_x = cv2.imread(os.path.join('Webcam/O',file))
               img_processada_x = process(img_x)

               template = img_processada

               # Template Matching to Detect Pattern
               result = cv2.matchTemplate(img_processada_x, template, cv2.TM_CCOEFF_NORMED)
               min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

               dx = int(max_loc[0] - max_loc[0])
               dy = int(max_loc[1] - max_loc[1])

               rows, cols = img_processada_x.shape
               M = np.float32([[1, 0, dx], [0, 1, dy]])
               aligned_image2 = cv2.warpAffine(img_processada_x, M, (cols, rows))

               # Subtract Images
               img_sub_processada = cv2.subtract(aligned_image2, img_processada)
               #img_sub = cv2.subtract(img_processada_x, img_processada)

               #img_sub_processada = process_symbols(img_sub)

               square = lista_quadrados[4]
               teste = np.array([[ponto[0],ponto[1]] for ponto in square])
               teste = teste.reshape((-1, 1, 2))
               cv2.polylines(img_sub_processada,[teste], True, (255, 0, 0), 2)
               cv2.imwrite(f"dataset/teste/primeira_subtraida_processada{file}.jpg", img_sub_processada)
               cv2.imwrite(f"dataset/teste/primeira_subtraida_processada2{file}.jpg", img)
               
               if lista_quadrados is None:
                    raise("Deu Ruim")
               #try:
               #     # pegando os contornos internos (quadrado do meio) e externos
               #     inner, outer = sorted(map(convex_hull, contours), key=len)
               #except:
               #     if f"Webcam/O/TabuleiroO{i}.jpg" not in defective_images:
               #          defective_images.append(f"Webcam/O/TabuleiroO{i}.jpg")
               #     continue
               # usando os contornos para calcular as coordenadas dos quadrados da matriz

               for i in range(0,9):
                    lista_imagens_cortadas = get_squares_init(np.array(lista_quadrados[i]))
                    cv2.imwrite(f"dataset/empty/o{index_image}.png",lista_imagens_cortadas)
                    imagem_cortada_com_x_tratada = get_squares_after_play(np.array(lista_quadrados[i]), img_sub_processada)
                    cv2.imwrite(f"dataset/o/o{index_image}.png",imagem_cortada_com_x_tratada)
                    index_image += 1


In [4]:
defective_images

[]

In [8]:
index_image = 0
defective_images = []
for f in range(1,14):
     img = cv2.imread(f"Webcam/X/TabuleiroX{f}.jpg")
     img_processada = process(img)
     contours, _ = cv2.findContours(img_processada, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE) #talvez usar o CHAIN_APPROX_SIMPLE, ver a diferenca em https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html
     aux = sorted(map(convex_hull, contours), key=len)
          
     lista_quadrados = get_square_corners_v2(aux)
     #for square in lista_quadrados:
     #square = lista_quadrados[4]
     #teste = np.array([[ponto[0],ponto[1]] for ponto in square])
     #teste = teste.reshape((-1, 1, 2))
     #cv2.polylines(img_sub_processada,[teste], True, (255, 0, 0), 2)
     #cv2.imwrite(f"dataset/teste/primeira_subtraida_processada{file}.jpg", img_sub_processada)
     #cv2.imwrite(f"dataset/teste/primeira_subtraida_processada2{file}.jpg", img)
     for file in os.listdir('Webcam/X/'):
          if file.find(f'X{f}-') != -1:
               img_x = cv2.imread(os.path.join('Webcam/X',file))
               img_processada_x = process(img_x)

               template = img_processada

               # Template Matching to Detect Pattern
               result = cv2.matchTemplate(img_processada_x, template, cv2.TM_CCOEFF_NORMED)
               min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

               dx = int(max_loc[0] - max_loc[0])
               dy = int(max_loc[1] - max_loc[1])

               rows, cols = img_processada_x.shape
               M = np.float32([[1, 0, dx], [0, 1, dy]])
               aligned_image2 = cv2.warpAffine(img_processada_x, M, (cols, rows))

               # Subtract Images
               img_sub_processada = cv2.subtract(aligned_image2, img_processada)

               #img_sub = cv2.subtract(img_processada_x, img_processada)

               #img_sub_processada = process_symbols(img_sub)
               #cv2.imwrite(f"dataset/teste/primeira_subtraida_processada{file}.jpg", img_processada)
               
               if lista_quadrados is None:
                    raise("Deu Ruim")
               for i in range(0,9):
                    lista_imagens_cortadas = get_squares_init(np.array(lista_quadrados[i]))
                    cv2.imwrite(f"dataset/empty/x{index_image}.png",lista_imagens_cortadas)
                    imagem_cortada_com_x_tratada = get_squares_after_play(np.array(lista_quadrados[i]), img_sub_processada)
                    cv2.imwrite(f"dataset/x/x{index_image}.png",imagem_cortada_com_x_tratada)
                    index_image += 1


In [14]:
index_image = 0
defective_images = []
for i in range(1,20):
     img = cv2.imread(f"Webcam/X/TabuleiroO{i}.jpg")
     img_processada = process(img)
     for file in os.listdir('Webcam/O/'):
          if file.find(f'O{i}-') != -1:
               img_x = cv2.imread(os.path.join('Webcam/O',file))
               img_processada_x = process(img_x)

               img_sub = cv2.subtract(img_processada_x, img_processada)

               img_sub_processada = process_symbols(img_sub)

               contours, _ = cv2.findContours(img_processada, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE) #talvez usar o CHAIN_APPROX_SIMPLE, ver a diferenca em https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html
               try:
                    # pegando os contornos internos (quadrado do meio) e externos
                    inner, outer = sorted(map(convex_hull, contours), key=len)
               except:
                    if f"Webcam/O/TabuleiroO{i}.jpg" not in defective_images:
                         defective_images.append(f"Webcam/O/TabuleiroO{i}.jpg")
                    continue
               # usando os contornos para calcular as coordenadas dos quadrados da matriz
               lista_quadrados = get_square_corners(inner,outer)

               for i in range(0,9):
                    k=i+1
                    if(k%3==0):
                         j=3
                    else:
                         j=k%3
                    k=int(i/3)+1
                    lista_imagens_cortadas = get_squares_init(np.array(lista_quadrados[i]))
                    cv2.imwrite(f"dataset/empty/b{index_image}.png",lista_imagens_cortadas)
                    imagem_cortada_com_x_tratada = get_squares_after_play(np.array(lista_quadrados[i]), img_sub_processada)
                    cv2.imwrite(f"dataset/o/o{index_image}.png",imagem_cortada_com_x_tratada)
                    if index_image == 161:
                         pass
                    index_image += 1


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
defective_images